In [5]:
!pip install --upgrade --force-reinstall git+https://github.com/pswpswpsw/nif.git

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/pswpswpsw/nif.git to /tmp/pip-req-build-o0er2qy4
  Running command git clone -q https://github.com/pswpswpsw/nif.git /tmp/pip-req-build-o0er2qy4
  Resolved https://github.com/pswpswpsw/nif.git to commit fd334c1319927041f469899a029ee86a5111ad73
     |████████████████████████████████| 11.5 MB 1.3 MB/s eta 0:00:01
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
     |████████████████████████████████| 3.1 MB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 247 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 12.0 MB/s eta 0:00:01
  Created wheel for NIF: filename=NIF-1.0.0-py

# How to training on large dataset (>5GB) where data cannot fit in memory?

### Note 

- GPU memory is precious, don't waste it on dataset
- It can be super interesting to train NIF on HUGE dataset!
- In this Google Colab, I will show a demonstration using [tensorflow record](https://www.tensorflow.org/tutorials/load_data/tfrecord),
which is a simple format for storing a sequence of binary records.
- First you will learn how to obtain `tfrecord` file from `npz`, then you will
learn how to perform training across different `tfrecord` files.

## 0. Prepare a "large" numpy `npz` to play with

- this "ficitious" dataset has 7 dimensions, 
$$(t,x,y,z,u,v,w)$$
where $u,v,w$ are three components of velocity field. 
- Thus, we have **4 features, and 3 targets, no weight**. We will need this information to create the dataset 

- let's call it `Big_npz_file.npz`, it has 54 MB with $10^6$ points, each point contains 7 real number values
- it has a key argument `data`, which stores the data


In [6]:
import numpy as np

data = np.random.uniform(0,1,(1000000,7))
np.savez('Big_npz_file.npz', data=data)

## 1. Generate `tfrecord` files from a big a `npz` 

First, you need to bring the `TFRDataset` class under `nif.data.tfr_dataset`

In [7]:
from nif.data.tfr_dataset import TFRDataset

Second, create an instance, a "file handler", call it `fh`, given the `n_features = 3` and `n_target = 3`

In [ ]:
fh = TFRDataset(n_feature=3, n_target=3)

### Then, you need to decide 
1. How many points to store in one single **tfrecord** file ----> `num_pts_per_file`
    - it doesn't have to be perfectly dividing the total number of points, we will make the last one smaller if it doesn't have a perfect divide.
    - Tensorflow official document [suggest](https://www.tensorflow.org/tutorials/load_data/tfrecord) around 100 MB.
    
2. What's the `npz` filepath? ----> `npz_path`
3. What's the key to access the 2-D matrix pointwise data? ----> `npz_key`
4. Where do you want to put the generated tfrecord file? ----> `tfr_path`
5. What's the prefix you want to put in the filename for the tfrecord file? 
    - they will be something like `prefix_0.npz`, ..., `prefix_10.npz`, ...

###  If you have answers to the above in your mind, now you can just call 
### `fh.create_from_npz(num_pts_per_file, npz_path, npz_key, write_tfr_path, prefix)`

In [ ]:
fh.create_from_npz(num_pts_per_file=5e6, npz_path='Big_npz_file.npz',
                   npz_key='data', tfr_path='TFR_dir',
                   prefix="case1")

### Now, you can see the **tfrecord** files that you generated! 

In [ ]:
%ls TFR_dir -lrth

# 2. How to train a Keras Model on the tfrecord files that generated above?

### Let's create a toy Model first

In [11]:
import tensorflow as tf 

x = tf.keras.Input(4,)
l1 = tf.keras.layers.Dense(3, activation='tanh')
model = tf.keras.Model([x], [l1(x)])
model.compile(optimizer='adam', loss='mse')

Next, we generate a **meta dataset** or maybe you can call it **hyper dataset**. We will do something called **sub-dataset-batching** (I called it). 

- First you need to determine how many epoch you want to run ----> `epoch`
- Then we call the method `.get_tfr_meta_dataset(path, epoch)`

In [ ]:
epoch = 100
meta_dataset = fh.get_tfr_meta_dataset(tfr_path='TFR_dir', epoch=epoch)

### Finally, we do sub-dataset batching with `meta_dataset`

- we obtain `batch_file`, which are Tensors basically, from meta_dataset
- we create a `sub` dataset from `batch_file`

- you can add your favourite callbacks if you want
- the default `epoch` in `model.fit` is 1, which makes sense in this context

In [ ]:
callbacks = []
batch_size = 128

for batch_file in meta_dataset:
    batch_dataset = fh.gen_dataset_from_batch_file(batch_file, batch_size)
    model.fit(batch_dataset, verbose=1, callbacks=callbacks)

### Cautious! 
- it is not perfect since we are `.fit` for each `tfrecord` and this will make epoch information lost after each recall of `.fit`
- we can hack it by using a `callbacks` that explicitly write loss to the disk.. in an append way.
- **again what is presented here is not perfect, I am open for your solutions!**

### Question:

- **Why are you choosing to chop tfrecord like this?** It seems to me it is not what the official document has suggested, the syntax here looks very not software-optimized. 

### **Answer:** 

- Because the standard way of using `tfrecord` will take each data points, i.e., $(t,x,y,z,u,v,w)$ as an `tf.train.Example`. When you have 10 of millions, hundreds of millsion data points, **generating such dataset (not using this dataset for training)** has been reported to have performance issues if your algorithm contains any native Python Loop, which is not avoidable at the current stage. 

- In `nif.data.tfr_dataset.TFRDataset`, we subsampling npz file in a sequential order and take data along each dimension (e.g., (t_0,...,t_M) ) as an `tf.train.Example`, instead of treating each single point `(t,x,y,z,u,v,w)` as an example. So we have 7 `Example` in this demo. 

- In this way, performance issue is greatly reduced. I believe the reason is that we leave the inner looping over all datapoints to the low-level

- So far, I can generate tfrecord from tens of Gigabyte data within 30 mins to 1 hour. While if I tried to make each training example as data point, it may take 

### Let me know <shawnpan@uw.edu>  if you have better solution for this, i.e., without using the meta-dataset to do sub-dataset-batching